# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from itertools import chain

import sqlalchemy
from sqlalchemy import create_engine

import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download(['stopwords','wordnet','punkt','averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\manor\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [3]:
print(f"df type: {type(df)}\nLen of df: {len(df)}")
print(f"Shape of X: {X.shape},\nShape of Y: {y.shape}")
print(list(df.columns))

df type: <class 'pandas.core.frame.DataFrame'>
Len of df: 10170
Shape of X: (10170,),
Shape of Y: (10170, 36)
['id', 'message', 'original', 'genre', 'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [4]:
# np.sum(df.isnull())

# # Check nulls
# 100 * (df.isnull().mean())

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Tokenize text into words
    words = list(chain.from_iterable([word_tokenize(t) for t in text]))
    
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words("english")]
    
    # Tag words with POS
    tagged_words = pos_tag(words)
    
    # Initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    # Iterate through each tagged word
    for word, tag in tagged_words:
        # Lemmatize based on POS tag
        if tag.startswith('N'):
            clean_tok = lemmatizer.lemmatize(word, pos='n').lower().strip()
        elif tag.startswith('V'):
            clean_tok = lemmatizer.lemmatize(word, pos='v').lower().strip()
        elif tag.startswith('J'):
            clean_tok = lemmatizer.lemmatize(word, pos='a').lower().strip()
        else:
            clean_tok = lemmatizer.lemmatize(word).lower().strip()
        
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [6]:
# tokenize(X)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# # Load the dataset
# engine = create_engine('sqlite:///DisasterResponse.db')
# df = pd.read_sql_table("DisasterResponse", engine)
# # X = df['message'].values
# # Y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values

# # Split the data into features (X) and labels (y)
# X = df['message']
# y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a pipeline for text classification
# pipeline = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize3)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

# # Train the model
# pipeline.fit(X_train, y_train)

# # Make predictions on the testing set
# y_pred = pipeline.predict(X_test)

# Iterate through each output category and calculate the F1 score, precision, and recall
for i, column in enumerate(y.columns):
    print(f"Category: {column}\n")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("------------------------")

Category: related

              precision    recall  f1-score   support

           0       0.64      0.30      0.41       846
           1       0.71      0.92      0.80      1660
           2       1.00      0.03      0.05        37

    accuracy                           0.70      2543
   macro avg       0.78      0.41      0.42      2543
weighted avg       0.69      0.70      0.66      2543

------------------------
Category: request

              precision    recall  f1-score   support

           0       0.75      0.90      0.82      1625
           1       0.74      0.48      0.58       918

    accuracy                           0.75      2543
   macro avg       0.75      0.69      0.70      2543
weighted avg       0.75      0.75      0.73      2543

------------------------
Category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2538
           1       0.00      0.00      0.00         5

    accuracy           

c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2453
           1       0.00      0.00      0.00        90

    accuracy                           0.96      2543
   macro avg       0.48      0.50      0.49      2543
weighted avg       0.93      0.96      0.95      2543

------------------------
Category: transport

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2480
           1       0.00      0.00      0.00        63

    accuracy                           0.98      2543
   macro avg       0.49      0.50      0.49      2543
weighted avg       0.95      0.98      0.96      2543

------------------------
Category: buildings

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2434
           1       1.00      0.01      0.02       109

    accuracy                           0.96      2543
   macro avg       0.98      0.50   

c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
def build_model():
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])
    # specify parameters for grid search
    parameters = {
        "vect__ngram_range":[(1,1),(1,2)],
        "clf__n_estimators": [10,50,100],
        "clf__min_samples_split":[2, 3, 4],
        "clf__criterion":["gini", "entropy"]
    }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [15]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\pipeline.py", line 753, in score
    return self.steps[-1][1].score(X

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:

# Iterate through each output category and calculate the F1 score, precision, and recall
for i, column in enumerate(y.columns):
    print(f"Category: {column}\n")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("------------------------")

Category: related

              precision    recall  f1-score   support

           0       0.53      0.45      0.48       846
           1       0.73      0.80      0.76      1660
           2       0.67      0.05      0.10        37

    accuracy                           0.67      2543
   macro avg       0.64      0.43      0.45      2543
weighted avg       0.66      0.67      0.66      2543

------------------------
Category: request

              precision    recall  f1-score   support

           0       0.72      0.89      0.80      1625
           1       0.67      0.40      0.50       918

    accuracy                           0.71      2543
   macro avg       0.70      0.65      0.65      2543
weighted avg       0.70      0.71      0.69      2543

------------------------
Category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2538
           1       0.00      0.00      0.00         5

    accuracy           

c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      2473
           1       0.00      0.00      0.00        70

    accuracy                           0.97      2543
   macro avg       0.49      0.50      0.49      2543
weighted avg       0.95      0.97      0.96      2543

------------------------
Category: other_aid

              precision    recall  f1-score   support

           0       0.85      0.99      0.91      2151
           1       0.21      0.01      0.02       392

    accuracy                           0.84      2543
   macro avg       0.53      0.50      0.47      2543
weighted avg       0.75      0.84      0.78      2543

------------------------
Category: infrastructure_related

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2453
           1       0.00      0.00      0.00        90

    accuracy                           0.96      2543
   macro avg       0.48

c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manor\Downloads\Data Scientist\uda_ds_venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.